# HuggingFace로 두 문장의 논리적 모순 분류하기

In [30]:
!pip install transformers datasets evaluate accelerate scikit-learn > /dev/null

In [31]:
import numpy as np
import random
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EvalPrediction,
)
import evaluate

## Dataset 준비

load_dataset("nyu-mll/glue", "mnli") 로 dataset을 불러옵니다.

- 학습 때는 `train` split만 활용하셔야 합니다. 나머지 split은 사용불가입니다.
- Validation data가 필요한 경우, `train` split에서 가져오셔야 합니다.

- **입력**: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
- **출력:** 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
    - **Entailment:** 두 문장에 논리적 모순이 없습니다.
    - **Neutral:** 두 문장은 논리적으로 관련이 없습니다.
    - **Contradiction:** 두 문장 사이에 논리적 모순이 존재합니다.

In [32]:
dataset = load_dataset("nyu-mll/glue", "mnli")
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [33]:
dataset.keys()

dict_keys(['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

`train` data를 한 번 살펴보겠습니다.

In [34]:
dataset['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

- 모델, 토크나이저 로딩

In [35]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [36]:
def preprocess_function(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

In [37]:
encoded_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["premise", "hypothesis", "idx"]
)

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

HuggingFace `datasets`로 불러온 dataset은 `train_test_split`으로 쉽게 쪼갤 수 있습니다.

다음은 각 split의 크기입니다.

In [39]:
# 원본 학습 데이터의 10%만 선택 (주요 변경점)
train_dataset_full = encoded_dataset["train"]
print(f"전체 학습 데이터 크기: {len(train_dataset_full)}")

# 전체 데이터의 10%만 랜덤하게 선택
subset_size = int(len(train_dataset_full) * 0.1)
subset_indices = random.sample(range(len(train_dataset_full)), subset_size)
train_dataset_subset = train_dataset_full.select(subset_indices)
print(f"10% 서브셋 학습 데이터 크기: {len(train_dataset_subset)}")

train_indices, val_indices = train_test_split(
    range(len(train_dataset_subset)),
    test_size=0.1,
    random_state=42
)

train_dataset = train_dataset_subset.select(train_indices)
val_dataset = train_dataset_subset.select(val_indices)

전체 학습 데이터 크기: 392702
10% 서브셋 학습 데이터 크기: 39270


In [40]:
# 평가를 위한 metric 로드
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [41]:
# 데이터 콜레이터 정의
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 학습 코드

다음은 위에서 구현한 Transformer를 imdb로 학습하는 코드를 구현합니다.
먼저 다음과 같이 학습 인자들을 정의합니다.

In [42]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",  # Colab에서는 wandb 등 필요없음
)

각각의 부분들은 이전 주차에서 배웠던 내용들을 설정하는 것에 불과하다는 것을 알 수 있습니다.
요약하면 다음과 같습니다:
- `epochs`: training data를 몇 번 반복할 것인지 결정합니다.
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지 결정합니다.
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지 결정합니다.
위의 부분들 이외에도 사소한 구현 요소들도 지정할 수 있습니다.

다음은 loss 이외의 평가 함수들을 구현하는 방법입니다.

In [43]:
# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-43-fd64b5556dfd>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


`evaluate` 또한 HuggingFace의 library로 다양한 평가 함수들을 제공하고 있습니다.
이번 실습의 경우, 감정 분석 문제는 분류 문제이기 때문에 정확도를 계산할 수 있습니다.
위와 같이 예측 결과(`pred`)와 실제 label(`labels`)가 주어졌을 때 정확도를 계산하는 것은 `evaluate`의 accuracy 함수로 구현할 수 있습니다.

마지막으로 위의 요소들을 종합하여 학습할 수 있는 `Trainer`를 구현합니다.

In [44]:
trainer.train()

# 학습된 모델로 실제 검증 데이터셋에서 평가
print("\n'validation_matched' 데이터셋에서 평가 중...")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.716500,0.672702,0.728037
2,0.511100,0.662182,0.745098
3,0.332200,0.841906,0.735931
4,0.211100,1.084647,0.740260
5,0.143200,1.366982,0.733639
6,0.107600,1.645166,0.741788
7,0.072000,1.769316,0.735931
8,0.046300,1.925672,0.741533
9,0.025800,2.067465,0.744843
10,0.017000,2.093417,0.744589



'validation_matched' 데이터셋에서 평가 중...


모델, training 인자, training과 validation data, 부가적인 평가 함수, 그리고 tokenizer를 넘겨주면 끝입니다.
별개로 early stopping과 같은 기능도 주석 친 부분과 같이 `callbacks`로 구현할 수 있으니 참고해주시길 바랍니다.

위와 같이 만든 `Trainer`는 다음과 같이 학습을 할 수 있습니다.

In [45]:
# validation_matched 데이터셋 전처리
validation_matched = encoded_dataset["validation_matched"]

보시다시피 training loss는 잘 떨어지는 반면, validation loss는 중간부터 쭉 올라가는 것을 볼 수 있습니다.
Overfitting이 일어났다고 볼 수 있습니다.

위와 같이 학습이 끝난 후 validation loss가 가장 낮은 모델을 가지고 test data의 성능을 평가하는 것은 다음과 같이 구현할 수 있습니다.

In [46]:
# validation_matched에서 평가
results = trainer.evaluate(validation_matched)
print(f"validation_matched 결과: {results}")
print(f"정확도: {results['eval_accuracy'] * 100:.2f}%")

validation_matched 결과: {'eval_loss': 0.6721426248550415, 'eval_accuracy': 0.7399898115129904, 'eval_runtime': 32.6652, 'eval_samples_per_second': 300.473, 'eval_steps_per_second': 9.398, 'epoch': 10.0}
정확도: 74.00%
